## Tutorial as a exploration

As the code is being written the tutorial/examples are being written. So there is a qulaity of exploration to this section. Let try to run the simples code possible with this package



In [45]:
from zeppy import ppipes
import time

the function `waitsome` will take wait for the time in `seconds`

In [46]:
def waitsome(seconds):
    """wait for some seconds"""
    time.sleep(seconds)
    return seconds

Let us call the function 5 times and see how long it takes

In [47]:
waitlist = [1, 2, 3, 2, 1]
starttime = time.time()
for seconds in waitlist:
    waitsome(seconds)
endtime = time.time()
print(f'time to run this = {(endtime-starttime):.3f} seconds')

time to run this = 9.016 seconds


it took 9.01 seconds to run -> pretty close 1+2+3+2+1 = 9 seconds
Let us try this again using the `parallelpipe` function

In [48]:
waitlist = [1, 2, 3, 2, 1] 
starttime = time.time()
result = ppipes.ipc_parallelpipe(waitsome, waitlist)    
endtime = time.time()
print(f'time to run this = {(endtime-starttime):.3f} seconds')
print(result)

time to run this = 3.385 seconds
[1, 2, 3, 2, 1]


It took about 3.39 seconds. This is because the `ipc_parallelpipe` ran the function 5 times too. But it ran it in parallel - simultanesouly
Now let us run it with `verbose=True`, so we can see what is going on under the hood.

In [49]:
waitlist = [1, 2, 3, 2, 1] 
starttime = time.time()
result = ppipes.ipc_parallelpipe(waitsome, waitlist, verbose=True)    
endtime = time.time()
print(f'time to run this = {(endtime-starttime):.3f} seconds')
print(f'result = {result}')

started worker 0
started worker 1
started worker 2
started worker 3
started worker 4
started sink
number of calculations = 5
running item: 0, in worker: 0
started vent
running item: 2, in worker: 1
running item: 3, in worker: 2
running item: 4, in worker: 3
running item: 1, in worker: 0
Total taken time for all calcs: 3283 msec
time to run this = 3.360 seconds
result = [1, 2, 3, 2, 1]


### The output tells you what is happening:

- it starts up 5 workers to run `waitsome` 5 times
- it starts up the sink. The sink collects the results form the workers
- Then it starts the ventilator that sends the 5 tasks to the workers
- the tasks are distributed evenly in no particular order between the workers
- notice that the first worker got it's task before the ventilator finished distributed all the tasks and printed `started ventilator`

Now let us try this with only 2 workers

In [50]:
waitlist = [1, 2, 3, 2, 1] 
starttime = time.time()
result = ppipes.ipc_parallelpipe(waitsome, waitlist, verbose=True, nworkers=2)    
endtime = time.time()
print(f'time to run this = {(endtime-starttime):.3f} seconds')
print(result)

started worker 0
started worker 1
started sink
number of calculations = 5
running item: 0, in worker: 0
started vent
running item: 2, in worker: 1
running item: 1, in worker: 0
running item: 3, in worker: 0
running item: 4, in worker: 1
Total taken time for all calcs: 5077 msec
time to run this = 5.166 seconds
[1, 2, 3, 2, 1]


Looks like `worker 0` made 3 runs, and `worker 1` made 2 runs. The whole thing too 5.14 secs. Still quicker than 9 seconds

This is fun. Can we do some energyplus runs now. Ha! hold on. Let us move slowly. Here is a different function to try

In [51]:
def wait_add(first, second):
    """wait for the sum of first and second. return the sum"""
    seconds = first + second
    return waitsome(seconds)    

How are we going to send two args to the function

In [52]:
waitlist = [(1, 0), (1, 1), (2, 1), (2, 0), (0, 1)] 
starttime = time.time()
result = ppipes.ipc_parallelpipe(wait_add, waitlist, verbose=True)    
endtime = time.time()
print(f'time to run this = {(endtime-starttime):.3f} seconds')
print(result)

started worker 0
started worker 1
started worker 2
started worker 3
started worker 4
started sink
number of calculations = 5
running item: 0, in worker: 0
started vent
running item: 2, in worker: 1
running item: 3, in worker: 2
running item: 4, in worker: 3
running item: 1, in worker: 0
time to run this = 3.353 seconds
[1, 2, 3, 2, 1]
Total taken time for all calcs: 3284 msec


How about a function that has named arguments. Let us try that